#### Installations

In order to work through this nootbook, you must first install pyulog and pandas.

+ Install pyulog:
~~~~
sudo apt-get install python-testresources
sudo pip install pyulog
~~~~

+ Install pandas:
~~~~
sudo apt-get install python-pandas
~~~~

#### Record a flight log
+ In QGroundControl, click on the "page and magnifying glass" symbol in the top left corner. Select **Mavlink Console**.
+ To start recording a new flight log, enter `reboot` into the command terminal.
+ To stop recording, enter`reboot` into the command terminal again.

#### Locate flight log
+ The `.ulg` files are stored in `/var/lib/mavlink-router/`


#### Convert .ulg file to .csv file
+ In this nootbook we will work with a pandas dataframe. In order to create a pandas dataframe of flight log data, we must first convert the .ulg file to .csv files. This can be done with the following command:
~~~~
ulog2csv <your-ulg-file>.ulg
~~~~

+ One .ulg file will be split into multiple .csv files, each containing specific flight infomation 
    + The unfiltered accelorometer data is stored in the 'sensor_combined' .csv file
    + The filtered accelorometer date is stored in the 'vehicle_local_position' .csv file

In [10]:
import pandas as pd
import numpy as np
import scipy.integrate


################################################################################################################
# HELPFUL FUNCTIONS
################################################################################################################
def get_times(timestamps):
    """
    Given a list of timestamps (in microseconds), returns a list of times (in seconds) starting at t = 0.
    
        Args:
            - timestamps = list of timestamps (microseconds)
    """
    start = timestamps[0]
    k = 1.0/1000000
    
    times = []
    for timestamp in timestamps:
        time = k * (timestamp - start)
        times.append(time)
        
    return times
        
def double_integrate(times, acceleration):
    """
    Given a list of times and the cooresponding accelerations at those time, returns a list of tuples (time, dist)
    containing the time and total change in distance up to that time.
        
        Args:
            - times = list of times
            - acceleration = list of accelerations (acceleration[i] should coorespond to the acceleration at times[i])
    """  
    # Integrate acceleration to get velocity
    velocity = scipy.integrate.cumtrapz(acceleration, x = times, initial = 0)
    # Integrate velocity to get distance
    distance = scipy.integrate.cumtrapz(velocity, x = times, initial = 0)
    
    out = []
    for i in range(len(times)):
        out.append((times[i], distance[i]))
        
    return out

def magnitude(vector):
    """
    Given a vector, returns its magnitude
        
        Args:
            - vector: list (arbitrary length)
    """
    if vector == []:
        return None
    
    total = 0
    for component in vector:
        total += component**2
    
    return total**0.5


################################################################################################################
# MAKE DATA FRAME
################################################################################################################
# Dataframe containing unfiltered state estimation data recorded by the IMU
unfiltered = pd.DataFrame.from_csv('flightlog/test1/00041-2018-07-20_10-31-49_sensor_combined_0.csv')
# Dataframe containing state estimation data that has been filtered by ekf
filtered = pd.DataFrame.from_csv('flightlog/test1/00041-2018-07-20_10-31-49_vehicle_local_position_0.csv')

################################################################################################################
# GET ACCELERATION DATA
################################################################################################################
# Unfiltered accleration data
unfiltered_ax = unfiltered['accelerometer_m_s2[0]'].tolist()
unfiltered_ay = unfiltered['accelerometer_m_s2[1]'].tolist()
unfiltered_az = unfiltered['accelerometer_m_s2[2]'].tolist()
# Unfiltered times
unfiltered_times = get_times(unfiltered.index.tolist())

# Filter with running average
N = 50 # window size
unfiltered_ax = np.convolve(unfiltered_ax, np.ones((N,))/N, mode='same')
unfiltered_ay = np.convolve(unfiltered_ay, np.ones((N,))/N, mode='same')
unfiltered_az = np.convolve(unfiltered_az, np.ones((N,))/N, mode='same')

# Filtered (ekf) accelorometer data
filtered_ax = filtered['ax'].tolist()
filtered_ay = filtered['ay'].tolist()
filtered_az = filtered['az'].tolist()
# Filtered times
filtered_times = get_times(filtered.index.tolist())

################################################################################################################
# DOUBLE INTEGRATE ACCELERATION TO GET DISTANCE
################################################################################################################
unfiltered_dx = double_integrate(unfiltered_times, unfiltered_ax)
unfiltered_dy = double_integrate(unfiltered_times, unfiltered_ay)
unfiltered_dz = double_integrate(unfiltered_times, unfiltered_az)

filtered_dx = double_integrate(filtered_times, filtered_ax)
filtered_dy = double_integrate(filtered_times, filtered_ay)
filtered_dz = double_integrate(filtered_times, filtered_az)

In [11]:
################################################################################################################
# FINAL DISTANCE VECTOR
################################################################################################################
unfiltered_vector = [unfiltered_dx[-1][1], unfiltered_dy[-1][1]]
filtered_vector = [filtered_dx[-1][1], filtered_dy[-1][1]]

print('Unfiltered: [dx, dy] = ' + str(unfiltered_vector) + '   magnitude = ' + str(magnitude(unfiltered_vector)))
print('Filtered: [dx, dy] = ' + str(filtered_vector) + '   magnitude = ' + str(magnitude(filtered_vector)))





Unfiltered: [dx, dy] = [-28.374227793509004, -34.982537339178883]   magnitude = 45.0430318869
Filtered: [dx, dy] = [1.0164657221955946, 1.6711484994104082]   magnitude = 1.95600098964
